In [7]:
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set_style("whitegrid")

%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


As Turing shows, we start from homogenous system, where chemicals react like  

A+B $\rightarrow$  C+D 

but smallest random distortions can lead us to standing-wave like pattern formation 


We will use following equations to describe pattern formation : 

1. Two Diffusion equations (actually , 4 for 2 dim )

$\frac{\partial a(x,t)}{\partial t} = D_{a}\frac{\partial^{2} a(x,t)}{\partial x^{2}} + R_{a}(a(x,t),b(x,t))$

$\frac{\partial b(x,t)}{\partial t} = D_{b}\frac{\partial^{2} b(x,t)}{\partial x^{2}} + R_{b}(a(x,t),b(x,t))$

D is diffusion coefficient and $R_{a}$ and $R_{b}$ are reaction terms.

Solution which A.Turing proposes is complex Furier Series, but we can simplify it to Gaussian : 
$$ a(x,t) = \frac{a_{0}}{\sqrt{2\pi(\sigma_{0}^{2} + 2 D_{a} t)}} \exp\left(-\frac{x^{2}}{2(\sigma_{0}^{2} +  2 D_{a} t)}\right)$$



However, for simulating reaction diffusion we need a PDE solver, so let's start with implementing [explicit finite-difference method](https://en.wikipedia.org/wiki/Finite_difference_method#Example:_The_heat_equation) 
Time derivative shall be approximated as 

$$
\frac{\partial a(x,t)}{\partial t} \approx \frac{1}{dt}(a_{x,t+1} - a_{x,t})
$$

And the spatial part of the derivative (Laplacian) can be derived from Taylor expansion of $a_{x+1}$ and $a_{x-1}$ as :

$$
\frac{\partial^{2} a(x,t)}{\partial x^{2}} \approx \frac{1}{dx^{2}}(a_{x+1,t} + a_{x-1,t} - 2a_{x,t})
$$

Putting it all together, for the diffusion part of the equation we get.

$$
a_{x,t+1} = a_{x,t} + dt\left(  \frac{D_{a}}{dx^{2}}(a_{x+1,t} + a_{x-1,t} - 2a_{x,t})  \right)
$$

In [1]:
def laplacian1D(a, dx):
    return (
        - 2 * a
        + np.roll(a,1,axis=0)  # .roll for periodic boundary conditions
        + np.roll(a,-1,axis=0)
    ) / (dx ** 2)

def laplacian2D(a, dx):
    return (
        - 4 * a
        + np.roll(a,1,axis=0) 
        + np.roll(a,-1,axis=0)
        + np.roll(a,+1,axis=1)
        + np.roll(a,-1,axis=1)
    ) / (dx ** 2)

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np

class BaseStateSystem:
    """
    Base object for "State System".

    We are going to repeatedly visualise systems which are Markovian:
    the have a "state", the state evolves in discrete steps, and the next
    state only depends on the previous state.
    """
    def __init__(self):
        raise NotImplementedError()

    def initialise(self):
        raise NotImplementedError()

    def initialise_figure(self):
        fig, ax = plt.subplots()
        return fig, ax

    def update(self):
        raise NotImplementedError()

    def draw(self, ax):
        raise NotImplementedError()

    def plot_time_evolution(self, filename, n_steps=30):
        """
        Creates a gif from the time evolution of a basic state syste.
        """
        self.initialise()
        fig, ax = self.initialise_figure()

        def step(t):
            self.update()
            self.draw(ax)
            return ax,

        anim = animation.FuncAnimation(fig, step, frames=np.arange(n_steps), interval=20, blit=False)
        anim.save(filename=filename, dpi=60, fps=10, writer='imagemagick')
        plt.close()
        
    def plot_evolution_outcome(self, filename, n_steps):
        """
        Evolves and save the outcome of evolving the system for n_steps
        """
        self.initialise()
        fig, ax = self.initialise_figure()
        
        for _ in range(n_steps):
            self.update()

        self.draw(ax)
        fig.savefig(filename)
        plt.close()


In [4]:
class OneDimensionalDiffusionEquation(BaseStateSystem):
    def __init__(self, D):
        self.D = D #diffusion coefficient
        self.width = 1000
        self.dx = 10 / self.width
        print("dx = ", self.dx)
        self.dt = 0.9 * (self.dx ** 2) / (2 * D) #from CFL condition dt <= dx^2 / (2D)
        print("dt = ", self.dt)
        self.steps = int(0.1 / self.dt)
        
    def initialise(self):
        self.t = 0
        self.X = np.linspace(-5,5,self.width)
        self.a = np.exp(-self.X**2)  #smooth localized concentration profile
        
    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()

    def _update(self):      
        La = laplacian1D(self.a, self.dx)
        delta_a = self.dt * (self.D * La)       
        self.a += delta_a
        
    def draw(self, ax):
        ax.clear()
        ax.plot(self.X,self.a, color="r")
        ax.set_ylim(0,1)
        ax.set_xlim(-5,5)
        ax.set_title("t = {:.2f}".format(self.t))
    
one_d_diffusion = OneDimensionalDiffusionEquation(D=1)

one_d_diffusion.plot_time_evolution("diffusion.gif")

MovieWriter imagemagick unavailable; using Pillow instead.


dx =  0.01
dt =  4.5e-05


![result](diffusion.gif)

For the reaction equations, I'm going to use the [FitzHugh–Nagumo equation](https://en.wikipedia.org/wiki/FitzHugh%E2%80%93Nagumo_model)

$R_a(a, b) = a - a^{3} - b + \alpha$

$R_{b}(a, b) = \beta (a - b)$

In [6]:
class ReactionEquation(BaseStateSystem):
    def __init__(self, Ra, Rb):
        self.Ra = Ra
        self.Rb = Rb
        self.dt = 0.01
        self.steps = int(0.1 / self.dt)
        
    def initialise(self):
        self.t = 0
        self.a = 0.1
        self.b = 0.7
        self.Ya = []
        self.Yb = []
        self.X = []
        
    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()

    def _update(self):      
        delta_a = self.dt * self.Ra(self.a,self.b)      
        delta_b = self.dt * self.Rb(self.a,self.b)      

        self.a += delta_a
        self.b += delta_b
        
    def draw(self, ax):
        ax.clear()
        
        self.X.append(self.t)
        self.Ya.append(self.a)
        self.Yb.append(self.b)

        ax.plot(self.X,self.Ya, color="r", label="A")
        ax.plot(self.X,self.Yb, color="b", label="B")
        ax.legend()
        
        ax.set_ylim(0,1)
        ax.set_xlim(0,5)
        ax.set_xlabel("t")
        ax.set_ylabel("Concentrations")
        


In [56]:
alpha, beta =  0.2, 5

def Ra(a,b): return a - a**3 - b + alpha
def Rb(a,b): return (a - b) * beta
    
one_d_reaction = ReactionEquation(Ra, Rb)
one_d_reaction.plot_time_evolution("reaction.gif", n_steps=50)


MovieWriter imagemagick unavailable; using Pillow instead.


![result](reaction.gif)

## Combining Together in 1d...

In [23]:
def random_initialiser(shape):
    return(
        np.random.normal(loc=0, scale=0.05, size=shape),
        np.random.normal(loc=0, scale=0.05, size=shape)
    )

class OneDimensionalRDEquations(BaseStateSystem):
    def __init__(self, Da, Db, Ra, Rb,
                 initialiser=random_initialiser,
                 width=1000, dx=1, 
                 dt=0.1, steps=1):
        
        self.Da = Da
        self.Db = Db
        self.Ra = Ra
        self.Rb = Rb
        
        self.initialiser = initialiser
        self.width = width
        self.dx = dx
        self.dt = dt
        self.steps = steps

    def initialise(self):
        self.t = 0
        self.a, self.b = self.initialiser(self.width)

    def _update(self): 
        a,b,Da,Db,Ra,Rb,dt,dx = (
            self.a, self.b,
            self.Da, self.Db,
            self.Ra, self.Rb,
            self.dt, self.dx
        )

        
        La = laplacian1D(a, dx)
        Lb = laplacian1D(b, dx)

        delta_a = dt * (Da * La + Ra(a,b))       
        a += delta_a

        delta_b = dt * (Db * Lb + Rb(a,b))       
        b += delta_b

    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()

    def draw(self, ax):
        ax.clear()
        ax.plot(self.a, color="r", label="A")
        ax.plot(self.b, color="b", label="B")
        ax.legend()
        ax.set_ylim(-1,1)
        ax.set_title("t = {:.2f}".format(self.t))


In [ ]:
Da, Db, alpha, beta = 1, 100, -0.005, 10
width = 100
dx = 1
dt = 0.001

oneDsys = OneDimensionalRDEquations(
    Da, Db, Ra, Rb, 
    width=width, dx=dx, dt=dt, 
    steps=100
)

oneDsys.plot_time_evolution("1Dsys.gif")

MovieWriter imagemagick unavailable; using Pillow instead.


![1dsys](1Dsys.gif)

## 2D case 

In [ ]:
class TwoDimensionalRDEquations(BaseStateSystem):
    def __init__(self, Da, Db, Ra, Rb,
                 initialiser=random_initialiser,
                 width=100, height = 100, dx=1, 
                 dt=0.1, steps=1):
        
        self.Da = Da
        self.Db = Db
        self.Ra = Ra
        self.Rb = Rb
        
        self.shape = (width, height)
        self.initialiser = initialiser
        self.width = width
        self.dx = dx
        self.dt = dt
        self.steps = steps

    def initialise(self):
        self.t = 0
        self.a, self.b = self.initialiser(self.shape)

    def _update(self): 
        a,b,Da,Db,Ra,Rb,dt,dx = (
            self.a, self.b,
            self.Da, self.Db,
            self.Ra, self.Rb,
            self.dt, self.dx
        )

        
        La = laplacian2D(a, dx)
        Lb = laplacian2D(b, dx)

        delta_a = dt * (Da * La + Ra(a,b))       
        a += delta_a

        delta_b = dt * (Db * Lb + Rb(a,b))       
        b += delta_b
        
    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()

    def draw(self, ax):
        ax[0].clear()
        ax[1].clear()

        ax[0].imshow(self.a, cmap='jet')
        ax[1].imshow(self.b, cmap='jet')
        
        ax[0].grid(False)
        ax[1].grid(False)
        
        ax[0].set_title("A, t = {:.2f}".format(self.t))
        ax[1].set_title("B, t = {:.2f}".format(self.t))

    def initialise_figure(self):
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
        return fig, ax
    
    def plot_time_evolution(self, filename, n_steps=30):
        self.initialise()
        fig, ax = self.initialise_figure()

        im_a = ax[0].imshow(self.a, cmap='jet', animated=True)
        im_b = ax[1].imshow(self.b, cmap='jet', animated=True)
        ax[0].set_title("A")
        ax[1].set_title("B")
        ax[0].grid(False)
        ax[1].grid(False)

        def step(t):
            self.update()
            im_a.set_array(self.a)
            im_b.set_array(self.b)
            ax[0].set_title(f"A, t = {self.t:.2f}")
            ax[1].set_title(f"B, t = {self.t:.2f}")
            return [im_a, im_b]

        anim = animation.FuncAnimation(
            fig, step, frames=np.arange(n_steps),
            interval=20, blit=True
        )
        anim.save(filename=filename, dpi=60, fps=10, writer='imagemagick')
        plt.close()


    

In [62]:
# def Ra(a,b): return a - a**3 - b + alpha
# def Rb(a,b): return (a - b) * beta

def Ra(a,b): return a - a**2 - b + alpha
def Rb(a,b): return (a - b) * beta

#Da, Db, alpha, beta = 1, 100, -0.05, 5
Da, Db, alpha, beta = 1, 100, -0.05, 5
dx = 1
dt = 0.001

twoDsys = TwoDimensionalRDEquations(
    Da, Db, Ra, Rb,  dx=dx, dt=dt, 
    steps=100
)

twoDsys.plot_time_evolution("2Dsys.gif", n_steps=100)

MovieWriter imagemagick unavailable; using Pillow instead.
